In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
%%cu
#include <stdio.h>
#define N 1024
  void add(int *a, int *b, int *c) {
  int tid = 0; 
  while (tid < N) {
    c[tid] = a[tid] + b[tid];
    tid += 1;
  }
  }
int main() {
  cudaEvent_t start, stop;
  cudaEventCreate(&start) ;
  cudaEventCreate(&stop) ;
  cudaEventRecord(start, 0) ;
  int a[N], b[N], c[N];
  for (int i=0; i<N; i++) {
    a[i] = i;
    b[i] = i * i;
  }
  add( a, b, c );
  for (int i=0; i<N; i++) {
  printf("%d + %d = %d\n", a[i], b[i], c[i] );
  }
  cudaEventRecord( stop, 0 );
  cudaEventSynchronize( stop );
  float czas;
  cudaEventElapsedTime( &czas, start, stop );
  printf( "Czas wykonania %3.2f ms\n", czas );
  cudaEventDestroy(start);
  cudaEventDestroy(stop);
  return 0;
  }

In [ ]:
%%cu
#include <stdio.h>
#define N 1024
__global__ void add(int *a, int *b, int *c) {
  int tid = blockIdx.x;
  if (tid < N)
  c[tid] = a[tid] + b[tid];
}

int main() {
  cudaEvent_t start, stop;
  cudaEventCreate(&start) ;
  cudaEventCreate( &stop ) ;
  cudaEventRecord( start, 0 ) ;
  int a[N], b[N], c[N];
  int *dev_a, *dev_b, *dev_c;
  cudaMalloc( (void**)&dev_a, N * sizeof(int) );
  cudaMalloc( (void**)&dev_b, N * sizeof(int) );
  cudaMalloc( (void**)&dev_c, N * sizeof(int) );
  for (int i=0; i<N; i++) {
  a[i] = -i;
  b[i] = i * i;
  }
  cudaMemcpy( dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice );
  cudaMemcpy( dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice );
  add<<<N,1024>>>( dev_a, dev_b, dev_c );
  cudaMemcpy( c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost );
  for (int i=0; i<N; i++) {
    printf( "%d + %d = %d\n", a[i], b[i], c[i] );
  }
  cudaEventRecord( stop, 0 );
  cudaEventSynchronize( stop );
  float czas;
  cudaEventElapsedTime( &czas, start, stop );
  printf( "Czas wykonania %3.2f ms\n", czas );
  cudaEventDestroy( start );
  cudaEventDestroy( stop );
  cudaFree( dev_a );
  cudaFree( dev_b );
  cudaFree( dev_c );
  return 0;
}

Uzsykane wyniki(ms):
CPU: 0.93, 0.76, 0.82, 0.63, 0.74, 0.71, 0.83, 0.74, 0.74, 0.81

GPU: 0.41, 0.52, 0.43, 0.46, 0.42, 0.48, 0.42, 0.45, 0.42, 0.43

Średnie przyspieszenie: 1,75
